# Dataloading experiments

This notebook is for experimenting with different dataloading implementations and strategies.

Some requirements:

- working with very large datasets of text (100s of GBs) split across multiple files
- need to be able to load data in distributed training across multiple GPUs (i.e. each GPU sees non-overlapping portion of the data)
- need to be able to load batches of data (i.e. [batch_size, seq_len])
- need to be able to load data in a streaming fashion, so that the whole dataset doesn't need to fit into memory at once
- needs to include tokenization

Desired properties:

- fast
- memory efficient
- minimizes time spent loading data, so that the GPU is never waiting for the data to load


## Huggingface Dataset with Pytorch IterableDataset and DataLoader

Since we alread use huggingface to get our datasets, let's try using huggingface's Dataset in combination with Pytorch's DataLoader.

In [1]:
from gollem.data.common import DATA_CACHE_DIR
from gollem.tokenizer import get_tokenizer

tinystories_dataset_id = "roneneldan/Tinystories"
tinystories_ds_path = DATA_CACHE_DIR / "tinystories" / "TinyStories_all_data"

tokenizer = get_tokenizer("gpt2")

In [2]:
from datasets import load_dataset_builder
from datasets import IterableDataset

ds_builder = load_dataset_builder(path=str(tinystories_ds_path))
for k, v in ds_builder.info.__dict__.items():
    print(k, v)


Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

description 
citation 
homepage 
license 
features {'story': Value(dtype='string', id=None), 'instruction': {'features': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'prompt:': Value(dtype='string', id=None), 'words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}, 'summary': Value(dtype='string', id=None), 'source': Value(dtype='string', id=None)}
post_processed None
supervised_keys None
builder_name json
dataset_name tiny_stories_all_data
config_name default
version 0.0.0
splits {'train': SplitInfo(name='train', num_bytes=6519810584, num_examples=4967871, shard_lengths=[400000, 400000, 400000, 400000, 400000, 400000, 400000, 400000, 400000, 400000, 400000, 400000, 167871], dataset_name='tiny_stories_all_data')}
download_checksums {'/Users/jonathon/code/gollem/gollem/data/datasets/tinystories/TinyStories_all_data/data00.json': {'num_bytes': 140424235, 'checksum': None}, '/Users/jonathon/code/gollem/gollem/data/datasets/tinystories/TinySt

In [3]:
from datasets import get_dataset_split_names

print(get_dataset_split_names(path=str(tinystories_ds_path)))

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

['train']


In [4]:
from datasets import load_dataset


ds = load_dataset(path=str(tinystories_ds_path), split="train")

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

In [5]:
ds

Dataset({
    features: ['story', 'instruction', 'summary', 'source'],
    num_rows: 4967871
})

In [6]:
len(ds)

4967871

In [7]:
from typing import cast


iter_ds = load_dataset(path=str(tinystories_ds_path), split="train", streaming=True)
iter_ds = cast(IterableDataset, iter_ds)
iter_ds.column_names

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

['story', 'instruction', 'summary', 'source']

In [16]:
from typing import Generator

import numpy as np

iter_ds = cast(IterableDataset, iter_ds)
# Strategy
# 1. Use hugginface IterableDataset to load batches of examples
# 2. tokenize each batch of examples on the fly
# 3. concatenate tokens to get batch of seq_len tokens with no padding (collecting next batch of examples as necessary)
# 4. repeat

# Questions:
# - how to make this work with distributed training?
#    - can we customize the number of shards even if they are different from number of files?
tokenizer = get_tokenizer("gpt2")

data_column = "story"
# columns_to_remove = [c for c in iter_ds.column_names if c != data_column]


def tokenize_batch(examples):
    return {"text": tokenizer.encode_batch(examples[data_column])}


# get batch from iter_ds
processed_ds = iter_ds.map(
    tokenize_batch, batched=True, batch_size=32, remove_columns=iter_ds.column_names
)
processed_ds = cast(IterableDataset, processed_ds)
print(processed_ds)

# get batch from batched_ds
batched_ds = processed_ds.batch(batch_size=32)


batch_size = 32
seq_len = 1024


def get_batch() -> Generator[np.ndarray, None, None]:
    buffer = [[] * batch_size]
    current_batch = np.full((batch_size, seq_len), 0, dtype=np.long)
    current_idxs = np.zeros(batch_size, dtype=np.int32)

    for batch in batched_ds:
        batch_lens = np.array([len(t) for t in batch["text"]])
        batch_idxs = seq_len - batch_lens
        end_idxs = current_idxs

        current_batch[current_idxs:end_idxs] = 


for i, batch in enumerate(batched_ds):
    print(i, len(batch["text"]))
    if i == 33:
        break

# get batch from batched_ds
print(batch)


IndentationError: expected an indented block after function definition on line 37 (577354952.py, line 41)